## Probabilistic Programming 2: Assignment

In this assignment, we will look at one of the examples in the lecture on the Gaussian distribution. Consider the following factor graph:

![](figures/ffg-PP2-assignment.png)

The variables $x$ and $y$ are Gaussian distributed:

$$\begin{align*}
x \sim&\ \mathcal{N}(\mu_x,\Sigma_x) \\
y \sim&\ \mathcal{N}(\mu_y,\Sigma_y) \, .
\end{align*}$$

Both are two-dimensional. The square node on the left is a vector addition operation, with $\xi_1$ representing the result. The middle square node is a multiplication operation with a clamped matrix $A = \begin{bmatrix} 2 & 0 \\ 0 & 1 \end{bmatrix}$ resulting in $\xi_2$. The right square node is a vector subtraction between $\xi_2$ and a clamped vector $b = \begin{bmatrix} 0 \\ 1 \end{bmatrix}$, producing the variable $z$.

In [ ]:
using Pkg
Pkg.activate("workspace/")
Pkg.instantiate();

### **1) What are the parameters of the message going out of the left square node?**

To answer this question, you'll need to specify a model in ForneyLab. The cell below already has a graph, some clamped parameters, the algorithm compilation and the algorithm execution code. Your job is add the two Gaussian variables $x$, $y$ and the resulting variable $ξ_1$ to the graph. You don't need to assign `:id`'s to these variables, you don't need `placeholder()`s and you don't have to specify a `PosteriorFactorization`.

You can add variables to a graph using the `@RV` macro. Use `@RV var ~ ...` to define a stochastic variable (one that follows a particular distribution) and `@RV var = ...` for a deterministic variable. Options for distributions include `GaussianMeanVariance`, `GaussianMeanPrecision`, `Gamma`, `Wishart`, `Beta`, `Bernoulli`, `Categorical` and `Dirichlet`.

In [2]:
using ForneyLab

# Start factor graph
g = FactorGraph()

# Set parameters
μ_x = [1.0, 1.0]
Σ_x = [1.0  0.0;
       0.0  1.0]
μ_y = [0.0, 0.0]
Σ_y = [2.0  0.0;
       0.0  1.0]

### BEGIN SOLUTION
# Add variables
@RV x ~ GaussianMeanVariance(μ_x, Σ_x)
@RV y ~ GaussianMeanVariance(μ_y, Σ_y)

# Vector addition
@RV ξ_1 = x + y
### END SOLUTION

# Compile algorithm
algo = messagePassingAlgorithm(ξ_1)
source_code = algorithmSourceCode(algo)
eval(Meta.parse(source_code))

# Execute inference
messages = Array{Message}(undef, 3)
step!(Dict(), Dict(), messages)
println("Outgoing "*string(messages[3]))

Outgoing Message: 𝒩(m=[1.00, 1.00], v=[[3.00, 0.00][0.00, 2.00]])



In [3]:
# Check your answer
@assert dims(messages[3].dist) == 2
@assert typeof(messages[3]) == Message{GaussianMeanVariance,Multivariate}
### BEGIN HIDDEN TESTS
m = messages[3].dist.params[:m]
@assert sum(abs.(m .- [1.,1.])) < 1e-3
### END HIDDEN TESTS

In [4]:
# Check your answer
@assert dims(messages[3].dist) == 2
@assert typeof(messages[3]) == Message{GaussianMeanVariance,Multivariate}
### BEGIN HIDDEN TESTS
V = messages[3].dist.params[:v]
@assert sum(abs.(V .- [3. 0.;0. 2.])) < 1e-3
### END HIDDEN TESTS

Suppose $\xi_1$ follows a Gaussian distribution $\xi_1 \sim \mathcal{N}(\mu_{\xi_1}, \Sigma_{\xi_1})$. The middle square node will perform a matrix multiplication $\xi_2 = A \cdot \xi_1$.

### **2) What are the parameters of the message going out of the middle square node?**

Add $ξ_1$ and $ξ_2$ as variables to the graph.

In [5]:
# Start factor graph
g = FactorGraph()

# Set parameters
μ_ξ_1 = [1.0, 1.0]
Σ_ξ_1 = [1.0  0.0;
         0.0  1.0]

# Transition matrix
A = [2. 0.;
     0. 1.]

### BEGIN SOLUTION
# Add variables
@RV ξ_1 ~ GaussianMeanVariance(μ_ξ_1, Σ_ξ_1)

# Vector multiplication
@RV ξ_2 = A*ξ_1
### END SOLUTION

# Compile algorithm
algo = messagePassingAlgorithm(ξ_2)
source_code = algorithmSourceCode(algo)
eval(Meta.parse(source_code))

# Execute inference
messages = Array{Message}(undef, 2)
step!(Dict(), Dict(), messages)
println("Outgoing "*string(messages[2]))

Outgoing Message: 𝒩(m=[2.00, 1.00], v=[[4.00, 0.00][0.00, 1.00]])



In [6]:
# Check your answer
@assert dims(messages[2].dist) == 2
@assert typeof(messages[2]) == Message{GaussianMeanVariance,Multivariate}
### BEGIN HIDDEN TESTS
m = messages[2].dist.params[:m]
@assert sum(abs.(m .- [2.,1.])) < 1e-3
### END HIDDEN TESTS

In [7]:
# Check your answer
@assert dims(messages[2].dist) == 2
@assert typeof(messages[2]) == Message{GaussianMeanVariance,Multivariate}
### BEGIN HIDDEN TESTS
V = messages[2].dist.params[:v]
@assert sum(abs.(V .- [4. 0.;0. 1.])) < 1e-3
### END HIDDEN TESTS

The right square node is a vector subtraction, producing $z = \xi_2 - b$. Suppose the marginal for $\xi_2 \sim \mathcal{N}(\mu_{\xi_2}, \Sigma_{\xi_2})$. 

### **3) What are the parameters for the message going out of the right node?**

Add $\xi_2$ and $z$ as variables to the graph.

In [8]:
# Start factor graph
g = FactorGraph()

# Set parameters
μ_ξ_2 = [1.0, 1.0]
Σ_ξ_2 = [1.0  0.0;
         0.0  1.0]

# Transition matrix
b = [0., 1.]

### BEGIN SOLUTION
# Add variables
@RV ξ_2 ~ GaussianMeanVariance(μ_ξ_2, Σ_ξ_2)

# Vector subtraction
@RV z = ξ_2 - b
### END SOLUTION

# Compile algorithm
algo = messagePassingAlgorithm(z)
source_code = algorithmSourceCode(algo)
eval(Meta.parse(source_code))

# Execute inference
messages = Array{Message}(undef, 2)
step!(Dict(), Dict(), messages)
println("Outgoing "*string(messages[2]))

Outgoing Message: 𝒩(m=[1.00, 0.00], v=[[1.00, 0.00][0.00, 1.00]])



In [9]:
# Check your answer
@assert dims(messages[2].dist) == 2
@assert typeof(messages[2]) == Message{GaussianMeanVariance,Multivariate}
### BEGIN HIDDEN TESTS
m = messages[2].dist.params[:m]
V = messages[2].dist.params[:v]
@assert sum(abs.(m .- [1., 0.])) < 1e-3
@assert sum(abs.(V .- [1. 0.;0. 1.])) < 1e-3
### END HIDDEN TESTS